In [1]:
from dataclasses import dataclass, field

DIRECTIONS = {
    "N": (0, -1),
    "S": (0, +1),
    "W": (-1, 0),
    "E": (+1, 0),
}
CHECKS = {
    "N": ((-1, -1), (+0, -1), (+1, -1)),
    "S": ((-1, +1), (+0, +1), (+1, +1)),
    "W": ((-1, -1), (-1, +0), (-1, +1)),
    "E": ((+1, -1), (+1, +0), (+1, +1)),
}

@dataclass
class Elf:
    position: tuple[int, int]
    directions: list[str] = field(init=False)
    alone: bool = field(init=False, default=False)
    
    def __post_init__(self):
        self.directions = list("NSWE")
    
    def move(self, grid):
        ox, oy = self.position
        around = set((x, y) for y in range(oy - 1, oy + 2) for x in range(ox - 1, ox + 2) if (x, y) != (ox, oy))
        self.alone = False
        if not (around & grid):
            self.alone = True
            return
        for direction in self.directions:
            next_direction = False
            for (cx, cy) in CHECKS[direction]:
                dx, dy = ox + cx, oy + cy
                if (dx, dy) in grid:
                    next_direction = True
                    break
            if next_direction:
                continue
            sx, sy = DIRECTIONS[direction]
            proposition = (ox + sx, oy + sy)
            return proposition
    
    def update(self):
        self.directions.append(self.directions.pop(0))

In [2]:
elves = []
with open("Day23.txt") as file:
    for y, row in enumerate(file):
        for x, col in enumerate(row.strip("\n")):
            if col == ".":
                continue
            elves.append(Elf((x, y)))
len(elves)

2750

In [3]:
def run(elves):
    grid = {elf.position for elf in elves}
    propositions = {}
    all_alone = True
    for elf in elves:
        if proposition := elf.move(grid):
            propositions.setdefault(proposition, []).append(elf)
        elf.update()
        all_alone &= elf.alone
    for position, moving_elves in propositions.items():
        if len(moving_elves) > 1:
            continue
        elf = moving_elves[0]
        elf.position = position
    return all_alone

In [4]:
%%time
from math import inf

for turn in range(10):
    run(elves)
minx, maxx, miny, maxy = inf, -inf, inf, -inf
for elf in elves:
    x, y = elf.position
    minx, maxx = min(minx, x), max(maxx, x)
    miny, maxy = min(miny, y), max(maxy, y)
((maxx - minx + 1) * (maxy - miny + 1)) - len(elves)

CPU times: user 159 ms, sys: 737 µs, total: 160 ms
Wall time: 160 ms


4138

In [5]:
%%time
turn = 10
while not (all_alone := run(elves)):
    turn += 1
turn + 1

CPU times: user 11.5 s, sys: 617 µs, total: 11.5 s
Wall time: 11.6 s


1010